In [1]:
import transformers
from unsloth import FastLanguageModel
from typing import Optional
import os
import torch
import random

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-19 14:52:53 __init__.py:190] Automatically detected platform cuda.


In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
                model_name="unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit",
                max_seq_length=32768,
                fast_inference=True
            )

FastLanguageModel.for_inference(model)

tokenized_prompt = tokenizer.encode(f"<｜begin▁of▁sentence｜><｜User｜>How many r's are in Strawberry?<｜Assistant｜>", return_tensors="pt", add_special_tokens=False).to(model.device)

#generated_text = model.generate(tokenized_prompt, max_new_tokens=100, do_sample=True, temperature=0.6, top_p=0.95)

#print(tokenizer.decode(generated_text[0]))

Unsloth: Switching from Unsloth dynamic quant to normal quant since
we do not yet support fast inference for unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit
==((====))==  Unsloth 2025.2.5: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.621 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/DeepSeek-R1-Distill-Qwen-1.5B-bnb-4bit with actual GPU utilization = 22.33%
Unsloth: Your GPU has CUDA compute capability 8.9 with VRAM = 23.62 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 32768. Num Sequences = 160.
Unsloth: vLLM's KV Cache can use up to 3.51 GB. Also swap space = 5 GB.
INFO 02-19 14:53:50 config

[W219 14:53:51.177053761 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 02-19 14:53:51 weight_utils.py:252] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 02-19 14:53:52 model_runner.py:1115] Loading model weights took 1.5365 GB
INFO 02-19 14:53:52 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 02-19 14:53:54 worker.py:267] Memory profiling takes 1.49 seconds
INFO 02-19 14:53:54 worker.py:267] the current vLLM instance can use total_gpu_memory (23.62GiB) x gpu_memory_utilization (0.22) = 5.27GiB
INFO 02-19 14:53:54 worker.py:267] model weights take 1.54GiB; non_torch_memory takes 0.09GiB; PyTorch activation peak memory takes 2.14GiB; the rest of the memory reserved for KV Cache is 1.50GiB.
INFO 02-19 14:53:54 executor_base.py:110] # CUDA blocks: 3522, # CPU blocks: 11702
INFO 02-19 14:53:54 executor_base.py:115] Maximum concurrency for 32768 tokens per request: 1.72x
INFO 02-19 14:53:55 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error oc

Capturing CUDA graph shapes: 100%|██████████| 23/23 [00:06<00:00,  3.29it/s]

INFO 02-19 14:54:02 model_runner.py:1562] Graph capturing finished in 7 secs, took 0.46 GiB
INFO 02-19 14:54:02 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 10.04 seconds


In [9]:
# Example parameters:
min_thinking_tokens = 700  # The minimum tokens that must occur between <think> and <\think>
max_output_tokens = 1000   # Total tokens we want to generate in this example
max_batch_tokens = 20     # Generate tokens in small batches
replacement_text = " Let's continue thinking deeply... "  # Replacement text if end is premature

# Encode the markers to get their token IDs.
start_think_token = tokenizer.encode("<think>", add_special_tokens=False)[0]
end_think_token = tokenizer.encode("<\think>", add_special_tokens=False)[0]
replacement_ids = tokenizer.encode(replacement_text, add_special_tokens=False)

# Initialize flags and counters.
is_thinking = False         # Are we currently in the thinking phase?
thinking_token_count = 0    # Number of tokens generated between <think> and <\think>
total_new_tokens = 0        # Overall new tokens generated

# Start with your prompt (already on the model’s device).
final_tokens = tokenized_prompt

while total_new_tokens < max_output_tokens:
    outputs = model.generate(
        input_ids=final_tokens,
        max_new_tokens=min(max_batch_tokens, max_output_tokens - total_new_tokens),
        do_sample=True,
        temperature=0.6,
        top_p=0.95,
        return_dict_in_generate=True,
        output_scores=True
    )
    # Extract only the new tokens (those beyond what we already had)
    new_tokens = outputs.sequences[:, final_tokens.shape[1]:]
    total_new_tokens += new_tokens.shape[1]

    # Process the newly generated tokens one-by-one.
    for token in new_tokens[0]:
        token_id = token.item()
        
        # Check for the start thinking marker.
        if token_id == start_think_token:
            is_thinking = True
            thinking_token_count = 0
            final_tokens = torch.cat([final_tokens, token.unsqueeze(0).unsqueeze(0)], dim=1)
            continue

        # Check for the end thinking marker.
        if token_id == end_think_token:
            if is_thinking and thinking_token_count < min_thinking_tokens:
                # Not enough tokens were produced during thinking.
                # Substitute the end marker with our replacement text.
                final_tokens = torch.cat(
                    [final_tokens, torch.tensor([replacement_ids]).to(final_tokens.device)], dim=1
                )
                # Update our count by the number of tokens we injected.
                thinking_token_count += len(replacement_ids)
            else:
                # Enough thinking tokens were generated; accept the end marker.
                final_tokens = torch.cat([final_tokens, token.unsqueeze(0).unsqueeze(0)], dim=1)
            is_thinking = False  # End the thinking phase.
            continue

        # If we’re in the thinking phase, update the token count.
        if is_thinking:
            thinking_token_count += 1

        # Append the current token as usual.
        final_tokens = torch.cat([final_tokens, token.unsqueeze(0).unsqueeze(0)], dim=1)
        
        # Optionally, break early if EOS is generated.
        if token_id == model.config.eos_token_id:
            break

    # Stop if we have reached the EOS token.
    if final_tokens[0, -1].item() == model.config.eos_token_id:
        break

# Decode the full generated token sequence.
output_text = tokenizer.decode(final_tokens[0], skip_special_tokens=True)
print(output_text)


<｜User｜>How many r's are in Strawberry?<｜Assistant｜><think>
I need to determine how many times the letter 'r' appears in the word "Strawberry."

First, I'll write down the word: S T R A W B E R R Y.

Next, I'll examine each letter individually to count the 'r's.

The first 'r' is in the third position.
The second 'r' is in the eighth position.
The third 'r' is in the ninth position.

Adding these up, there are three 'r's in total.
</think>

To determine how many times the letter **'r'** appears in the word "Strawberry," let's break it down step by step:

1. **Write down the word:**  
   **Strawberry**

2. **Identify each letter and count the 'r's:**  
   - **S**  
   - **T**  
   - **R** (1st 'r')  
   - **A**  
   - **W**  
   - **B**  
   - **E**  
   - **R** (2nd 'r')  
   - **R** (3rd 'r')  
   - **Y**

3. **Count the total number of 'r's:**  
   - **1st 'r'**  
   - **2nd 'r'**  
   - **3rd 'r'**

   **Total:** **3**

\boxed{3}
